In [1]:
import digitalhub as dh

In [6]:
project = dh.get_or_create_project("project-dbt-ci")

In [7]:
print(project)

{'kind': 'project', 'metadata': {'project': 'project-dbt-ci', 'name': 'project-dbt-ci', 'created': '2024-10-23T13:14:02.867Z', 'updated': '2024-10-23T13:14:02.867Z', 'created_by': 'khurshid@fbk.eu', 'updated_by': 'khurshid@fbk.eu'}, 'spec': {'context': './', 'functions': [], 'artifacts': [], 'workflows': [], 'dataitems': [], 'models': []}, 'status': {'state': 'CREATED'}, 'user': 'khurshid@fbk.eu', 'id': 'project-dbt-ci', 'name': 'project-dbt-ci', 'key': 'store://project-dbt-ci'}


In [8]:
url = "https://gist.githubusercontent.com/kevin336/acbb2271e66c10a5b73aacf82ca82784/raw/e38afe62e088394d61ed30884dd50a6826eee0a8/employees.csv"
di = project.new_dataitem(name="employees-dbt",
                          kind="table",
                          path=url)

In [9]:
sql = """
WITH tab AS (
    SELECT  *
    FROM    {{ ref('employees') }}
)
SELECT  *
FROM    tab
WHERE   tab."DEPARTMENT_ID" = '50'
"""

In [10]:
function = project.new_function(name="function-dbt",
                                kind="dbt",
                                code=sql)

In [25]:
run = function.run("transform",
                   inputs={"employees": di.key},
                   outputs={"output_table": "department-50"})

In [12]:
print(run.refresh().status)

{'state': 'RUNNING', 'message': 'job j-dbttransform-17ceb79c-9449-444b-8397-59273c498e74 created', 'transitions': [{'status': 'RUNNING', 'message': 'job j-dbttransform-17ceb79c-9449-444b-8397-59273c498e74 created', 'time': '2024-10-23T13:14:24.08321968Z'}, {'status': 'READY', 'time': '2024-10-23T13:14:24.042246908Z'}, {'status': 'BUILT', 'time': '2024-10-23T13:14:24.019606525Z'}], 'k8s': {'secret': {'secret': {'apiVersion': 'v1', 'kind': 'Secret', 'metadata': {'labels': {'dhcore/framework': 'k8sjob', 'app.kubernetes.io/managed-by': 'dhcore', 'app.kubernetes.io/part-of': 'dhcore-project-dbt-ci', 'dhcore/project': 'project-dbt-ci', 'dhcore/runtime': 'dbt', 'app.kubernetes.io/instance': 'dhcore-17ceb79c-9449-444b-8397-59273c498e74', 'app.kubernetes.io/version': '17ceb79c-9449-444b-8397-59273c498e74'}, 'name': 'sec-dbttransform-17ceb79c-9449-444b-8397-59273c498e74', 'namespace': 'digitalhub-tenant1'}}}, 'job': {'job': {'apiVersion': 'batch/v1', 'kind': 'Job', 'metadata': {'creationTimestam

In [13]:
proj = dh.get_or_create_project("project-dbt-ci")
di = proj.get_dataitem('department-50')
df = di.as_df()
df.head()

,EMPLOYEE_ID,FIRST_NAME,LAST_NAME,EMAIL,PHONE_NUMBER,HIRE_DATE,JOB_ID,SALARY,COMMISSION_PCT,MANAGER_ID,DEPARTMENT_ID
0,198,Donald,OConnell,DOCONNEL,650.507.9833,21-JUN-07,SH_CLERK,2600,-,124,50
1,199,Douglas,Grant,DGRANT,650.507.9844,13-JAN-08,SH_CLERK,2600,-,124,50
2,120,Matthew,Weiss,MWEISS,650.123.1234,18-JUL-04,ST_MAN,8000,-,100,50
3,121,Adam,Fripp,AFRIPP,650.123.2234,10-APR-05,ST_MAN,8200,-,100,50
4,122,Payam,Kaufling,PKAUFLIN,650.123.3234,01-MAY-03,ST_MAN,7900,-,100,50


In [31]:
import os
new_folder = 'src'
if not os.path.exists(new_folder):
    os.makedirs(new_folder)

In [32]:
%%writefile "src/dbt_ci_pipeline.py"

from digitalhub_runtime_kfp.dsl import pipeline_context

def myhandler(url):
    with pipeline_context() as pc:
        s1_dataset = pc.step(name="dbt", function="function-dbt", action="transform", inputs={"employees":url}, outputs={"output_table": "department-50"})

Overwriting src/dbt_ci_pipeline.py


In [27]:
workflow = proj.new_workflow(name="pipeline_dbt", kind="kfp", code_src= "src/dbt_ci_pipeline.py", handler = "myhandler")

In [28]:
di_url = proj.new_dataitem(name="url_data_item",kind="table",path=url)

In [29]:
workflow_run = workflow.run(parameters={"url": di_url.key})